In [1]:
import tensorflow as tf
import keras
import numpy as np

In [2]:
from keras.datasets import mnist
(train_images, train_labels),(test_images, test_labels) = mnist.load_data()

In [3]:
# Scaling and resizing the inputs
X_train = train_images.reshape(train_images.shape[0],28*28)
y_train = train_labels

X_test = test_images.reshape(test_images.shape[0],28*28)
y_test = test_labels

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [4]:
from keras.layers import Dense , Input, Dropout
from keras.models import Model

In [5]:
inputs = Input(shape=(28*28,))

hidden_layer1 = Dense(512,activation="relu")(inputs)
hidden_layer1 = Dropout(0.5)(hidden_layer1)

hidden_layer2 = Dense(64,activation="relu")(hidden_layer1)

outputs = Dense(10,activation="softmax")(hidden_layer2)

model = Model(inputs=inputs,outputs=outputs)


In [6]:
# Custom Training Loop

In [7]:
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import RMSprop
from keras.metrics import SparseCategoricalAccuracy
from keras.metrics import Mean

In [8]:
loss_func = SparseCategoricalCrossentropy()
optimizer = RMSprop()
metrics = SparseCategoricalAccuracy()
loss_tracking_metric = Mean()

In [9]:
def custom_training(model,train_images,train_labels,num_epochs,batch_size):

# Creating a tensorflow dataset where each element corresponds to a pair of an image and its corresponding label.
    training_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels)) 
    training_dataset = training_dataset.batch(batch_size) # Batching the dataset 

# Epoch run for each training epoch
    for epoch in range(num_epochs):
        print(f"Starting of Epoch : {epoch}")
        #Iterating in a random batch of inputs
        for image_batch,label_batch in training_dataset:
            # Forward propogation starts form here
            with tf.GradientTape() as tape:  # Used to record the gradients for forward propagation so that we can use it during back propagation
                y_pred = model(image_batch)
                loss = loss_func(label_batch,y_pred)
            
            gradients = tape.gradient(loss,model.trainable_weights) # Recoding the gradients of loss wrt to the model trainable weights
            optimizer.apply_gradients(zip(gradients,model.trainable_weights)) # Applying the gradients to optimize the trainable weights
            metrics.update_state(label_batch,y_pred) # To calculate the accuracy of the epoch

        train_accuracy = metrics.result() # Recording the accuracy and displaying the results
        print(f"Accuracy in Epoch {epoch} : {train_accuracy}")
        metrics.reset_states()
            


In [10]:
custom_training(model,X_train,y_train,5,128)

Starting of Epoch : 0
Accuracy in Epoch 0 : 0.9247833490371704
Starting of Epoch : 1
Accuracy in Epoch 1 : 0.9708333611488342
Starting of Epoch : 2
Accuracy in Epoch 2 : 0.9818166494369507
Starting of Epoch : 3
Accuracy in Epoch 3 : 0.9874333143234253
Starting of Epoch : 4
Accuracy in Epoch 4 : 0.991183340549469


In [17]:
@tf.function # Used to run the function faster, but use this once our function is debugged and running fine(can be used in training as well)
def custom_evaluation(model,test_images,test_labels):

    # Creating a tensorflow dataset where each element corresponds to a pair of an image and its corresponding label.
    test_dataset = tf.data.Dataset.from_tensor_slices((test_images,test_labels)) 
    test_dataset = test_dataset.batch(128) # Batching the dataset

    for image_batch,label_batch in test_dataset:
        y_pred = model(image_batch,training=True)
        loss = loss_func(label_batch,y_pred)
        metrics.update_state(label_batch,y_pred)

    test_accuracy = metrics.result()
    print(f"Accuracy on test set : {test_accuracy}")
    metrics.reset_states()


In [18]:
history = custom_evaluation(model,X_test,y_test)
history

Accuracy on test set : Tensor("Identity:0", shape=(), dtype=float32)
